In [77]:
import pandas as pd
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

# print(x_train)
# print(x_train.info(),x_test.info())
df = pd.concat([x_train,y_train.iloc[:,1]],axis=1)
# display(df)
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
# scaler = MinMaxScaler()
# df['Sc_Credit'] = scaler.fit_transform(df[['CreditScore']])
# print(scaler.fit_transform(df[['CreditScore']]))
# print(df[['Sc_Credit','Exited']].corr())
# print(df['Sc_Credit'])

# print(df.corr())
# print(df)
df.drop(['CustomerId','CreditScore','Tenure','NumOfProducts','HasCrCard','EstimatedSalary'],axis=1,inplace=True)
# print(df)
df.drop('Surname',axis=1,inplace=True)
# print(df)
# print(df.groupby(['Geography','Exited'])['Exited'].count())
df['Geography'] = df['Geography'].replace('Germany',1).replace(['France','Spain'],0)
# print(df)
df['Gender'] = df['Gender'].replace('Female',0).replace('Male',1)
# print(df)
# print(df['Age'].describe())
df.loc[df['Age']>56,'Age'] = 56
# print(df['Age'].describe())
df['Age'] = scaler.fit_transform(df[['Age']])
# print(df)
df.loc[df['Balance']>0,'Balance'] = 1
# print(df.groupby(['Balance','Exited'])['Exited'].size())
# print(df)
x_test_id = x_test['CustomerId']
x_train.drop(['Surname','CustomerId','CreditScore','Tenure','NumOfProducts','HasCrCard','EstimatedSalary'],axis=1,inplace=True)
x_test.drop(['Surname','CustomerId','CreditScore','Tenure','NumOfProducts','HasCrCard','EstimatedSalary'],axis=1,inplace=True)
x_train['Geography'] = x_train['Geography'].replace('Germany',1).replace(['France','Spain'],0)
x_test['Geography'] = x_test['Geography'].replace('Germany',1).replace(['France','Spain'],0)
x_train['Gender'] = x_train['Gender'].map(lambda x: x.strip().lower()).replace('female',0).replace('male',1)
x_test['Gender'] = x_test['Gender'].map(lambda x:x.strip().lower()).replace('female',0).replace('male',1)
x_train.loc[x_train['Age']>56,'Age'] = 56
x_test.loc[x_test['Age']>56,'Age'] = 56
x_train['Age'] = scaler.transform(x_train[['Age']])
x_test['Age'] = scaler.transform(x_test[['Age']])
x_train.loc[x_train['Balance']>0,'Balance'] = 1
x_test.loc[x_test['Balance']>0,'Balance'] = 1
y_train.drop('CustomerId',axis=1,inplace=True)
# print(y_train)
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000,max_depth=3)
model.fit(x_train,y_train.values.ravel())
x_val_predicted = model.predict(x_val)
from sklearn.metrics import roc_auc_score
print(roc_auc_score(x_val_predicted,y_val))
result = pd.DataFrame(model.predict(x_test),columns=['Exited'])
final = pd.concat([x_test_id,result],axis=1)
final.to_csv('12345.csv',index=False)

y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv')
final = pd.read_csv('12345.csv',index_col=0)
print(roc_auc_score(final['Exited'],y_test.iloc[:,1]))

0.8312932169281748
0.800179211469534
